In [1]:
import pandas as pd
movie_test = "Pelis_short.csv"
movie_ratings = "Movie_Ratings.csv"
from KNN_Recommender import * 

In [2]:
#from Knn import KNNCalcularDistancia
from backend.distances import euclidean_distance, cosine_similarity, manhattan_formulas, pearson

In [3]:
def getDataFrameAndUsers(df_selected):
    df = pd.read_csv(df_selected, index_col=0)
    users = df.columns
    return df, users


In [41]:
df, users =  getDataFrameAndUsers(movie_ratings)

In [42]:
df

,Patrick C,Heather,Bryan,Patrick T,Thomas,aaron,vanessa,greg,brian,ben,...,Zak,Matt,Chris.1,Josh,Amy,Valerie,Gary,Stephen,Jessica,Jeff
Alien,NaN,NaN,2.0,NaN,5.0,4.0,NaN,NaN,4.0,NaN,...,NaN,NaN,4.0,3.0,NaN,NaN,2.0,5.0,NaN,4.0
Avatar,4.0,5.0,5.0,4.0,2.0,NaN,4.0,3.0,NaN,3.0,...,5.0,NaN,NaN,4.0,3.0,2.0,1.0,4.0,NaN,4.0
Blade Runner,5.0,NaN,NaN,NaN,5.0,4.0,NaN,1.0,5.0,5.0,...,NaN,NaN,3.0,NaN,3.0,3.0,1.0,NaN,NaN,5.0
Braveheart,4.0,NaN,5.0,NaN,4.0,4.0,3.0,4.0,4.0,NaN,...,5.0,NaN,4.0,NaN,3.0,4.0,5.0,5.0,NaN,4.0
Dodgeball,5.0,4.0,3.0,2.0,4.0,NaN,4.0,5.0,3.0,4.0,...,3.0,NaN,3.0,NaN,4.0,3.0,4.0,3.0,NaN,3.0
Forest Gump,4.0,5.0,4.0,3.0,3.0,NaN,4.0,5.0,5.0,5.0,...,5.0,4.0,5.0,4.0,4.0,5.0,4.0,1.0,4.0,4.0
Gladiator,NaN,5.0,5.0,NaN,4.0,4.0,4.0,5.0,5.0,5.0,...,5.0,NaN,3.0,4.0,NaN,3.0,3.0,4.0,NaN,4.0
Jaws,NaN,5.0,4.0,NaN,4.0,NaN,2.0,3.0,3.0,1.0,...,NaN,NaN,5.0,5.0,3.0,2.0,4.0,2.0,3.0,5.0
Kazaam,2.0,NaN,3.0,5.0,2.0,NaN,NaN,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0
Lord of the Rings,4.0,4.0,3.0,2.0,NaN,5.0,2.0,3.0,5.0,2.0,...,4.0,1.0,4.0,NaN,NaN,5.0,3.0,5.0,3.0,3.0


In [6]:
df[df['Patrick C'].isna()].index

Index(['Alien', 'Gladiator', 'Jaws', 'Pootie Tang', 'Pulp Fiction', 'Scarface',
       'Village'],
      dtype='object')

In [43]:
class KNNCalcularDistancia:
    def __init__(self, distance_function):
        self.distance_function = distance_function

    def calculate_distances(self, df: pd.DataFrame, target_column: str) -> pd.Series:
        if target_column not in df.columns:
            raise ValueError(f"La columna '{target_column}' no existe en el DataFrame")

        distances = {}
        target_series = pd.to_numeric(df[target_column], errors='coerce')

        for column in df.columns:
            if column == target_column:
                continue

            other_series = pd.to_numeric(df[column], errors='coerce')

            # Filtrar valores válidos (ambos no nulos)
            valid_pairs = [
                (x, y)
                for x, y in zip(target_series, other_series)
                if pd.notna(x) and pd.notna(y)
            ]
            if not valid_pairs:
                distances[column] = float('inf')  # o None
                continue

            values1, values2 = zip(*valid_pairs)
            try:
                distance = self.distance_function(list(values1), list(values2))
            except Exception:
                distance = float('inf')

            distances[column] = distance

        return pd.Series(distances).sort_values()

    def get_knn(self, df: pd.DataFrame, target_column: str, k: int = 5) -> list[dict]:
        distances = self.calculate_distances(df, target_column)
        knn = distances.head(k)

        return [
            {"neighbor": name, "distance": round(dist, 4)}
            for name, dist in knn.items()
            if not math.isinf(dist)
        ]
    
    def recommend(self, df: pd.DataFrame, target_column:str, neighbor:int=5, umbral:float=36)->list[dict]:
        # Paso 1: obtener KNN
        knn_df = self.get_knn(df, target_column, neighbor)

        neighbor_cols = [d['neighbor'] for d in knn_df]

        # Paso 2: identificar los índices donde el usuario no ha calificado (NaN)
        indices_nan = df[df[target_column].isna()].index

        recommendations = []

        for idx in indices_nan:
        # Calificaciones de neighbor en ese ítem
            neighbor_values = df.loc[idx, neighbor_cols]
            neighbor_values = neighbor_values.dropna()  # quitar NaNs

            if neighbor_values.empty:
                continue  # ningún neighbor calificó esta película

            promedio = neighbor_values.mean()

            if promedio >= umbral:
                # Construimos el dict del formato requerido
                recomendacion = {
                    'movie': idx,
                    'score': round(promedio, 3),
                    'neighbor': sorted(
                        [
                            {'name': neighbor, 'score': round(df.loc[idx, neighbor], 3)}
                            for neighbor in neighbor_values.index
                        ],
                        key=lambda x: x['score'],
                        reverse=True
                    )
                }
                recommendations.append(recomendacion)

        # Ordenar recommendations por score descendente
        recommendations.sort(key=lambda x: x['score'], reverse=True)
        return recommendations

In [44]:
knn_calc = KNNCalcularDistancia(manhattan_formulas.manhattan_distance)

In [45]:
knn_calc.get_knn(df, 'Patrick C', 6)

[{'neighbor': 'Josh', 'distance': 2.0},
 {'neighbor': 'Matt', 'distance': 7.0},
 {'neighbor': 'Katherine', 'distance': 7.0},
 {'neighbor': 'vanessa', 'distance': 8.0},
 {'neighbor': 'Jessica', 'distance': 9.0},
 {'neighbor': 'Jonathan', 'distance': 9.0}]

In [10]:
df['Patrick C']

Alien                   NaN
Avatar                  4.0
Blade Runner            5.0
Braveheart              4.0
Dodgeball               5.0
Forest Gump             4.0
Gladiator               NaN
Jaws                    NaN
Kazaam                  2.0
Lord of the Rings       4.0
Napolean Dynamite       3.0
Old School              3.0
Pootie Tang             NaN
Pulp Fiction            NaN
Scarface                NaN
Shawshank Redemption    5.0
Snakes on a Plane       4.0
Spiderman               4.0
Star Wars               5.0
The Dark Knight         4.0
The Happening           1.0
The Matrix              3.0
Toy Story               4.0
Village                 NaN
You Got Mail            1.0
Name: Patrick C, dtype: float64

In [11]:
df

,Patrick C,Heather,Bryan,Patrick T,Thomas,aaron
Alien,NaN,NaN,2.0,NaN,5.0,4.0
Avatar,4.0,5.0,5.0,4.0,2.0,NaN
Blade Runner,5.0,NaN,NaN,NaN,5.0,4.0
Braveheart,4.0,NaN,5.0,NaN,4.0,4.0
Dodgeball,5.0,4.0,3.0,2.0,4.0,NaN
Forest Gump,4.0,5.0,4.0,3.0,3.0,NaN
Gladiator,NaN,5.0,5.0,NaN,4.0,4.0
Jaws,NaN,5.0,4.0,NaN,4.0,NaN
Kazaam,2.0,NaN,3.0,5.0,2.0,NaN
Lord of the Rings,4.0,4.0,3.0,2.0,NaN,5.0


In [16]:
df[df['Patrick C'].isna()].index

Index(['Alien', 'Gladiator', 'Jaws', 'Pootie Tang', 'Pulp Fiction', 'Scarface',
       'Village'],
      dtype='object')

In [46]:
knn_calc.recommend(df, 'Patrick C', 5, 2.01)

[{'movie': 'Pulp Fiction',
  'score': np.float64(4.0),
  'neighbor': [{'name': 'Jessica', 'score': np.float64(4.0)}]},
 {'movie': 'Scarface',
  'score': np.float64(4.0),
  'neighbor': [{'name': 'vanessa', 'score': np.float64(4.0)}]},
 {'movie': 'Gladiator',
  'score': np.float64(3.667),
  'neighbor': [{'name': 'Josh', 'score': np.float64(4.0)},
   {'name': 'vanessa', 'score': np.float64(4.0)},
   {'name': 'Katherine', 'score': np.float64(3.0)}]},
 {'movie': 'Jaws',
  'score': np.float64(3.333),
  'neighbor': [{'name': 'Josh', 'score': np.float64(5.0)},
   {'name': 'Jessica', 'score': np.float64(3.0)},
   {'name': 'vanessa', 'score': np.float64(2.0)}]},
 {'movie': 'Alien',
  'score': np.float64(3.0),
  'neighbor': [{'name': 'Josh', 'score': np.float64(3.0)}]},
 {'movie': 'Village',
  'score': np.float64(3.0),
  'neighbor': [{'name': 'Josh', 'score': np.float64(4.0)},
   {'name': 'vanessa', 'score': np.float64(2.0)}]}]